In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# Problem set-up

In [2]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .4
l = .3
mu = .2
g = 10.
h = .05

In [3]:
# symbolic state
xb, yb, tb = sp.symbols('xb yb tb') # position of the ball
xf, yf = sp.symbols('xf yf') # position of the floor
xdb, ydb, tdb = sp.symbols('xdb ydb tdb') # velocity of the ball
xdf, ydf = sp.symbols('xdf ydf') # velocity of the floor
x = sp.Matrix([
    xb, yb, tb,
    xf, yf,
    xdb, ydb, tdb,
    xdf, ydf
])

# symbolic input
xd2f, yd2f = sp.symbols('xd2f yd2f') # acceleration of the floor
ftf, fnf = sp.symbols('ftf fnf') # floor force
ftc, fnc = sp.symbols('ftc fnc') # ceiling force
u = sp.Matrix([
    xd2f, yd2f,
    ftf, fnf,
    ftc, fnc
])

In [4]:
# ball velocity update
xdb_next = xdb + h*ftf/m - h*ftc/m
ydb_next = ydb + h*fnf/m - h*fnc/m - h*g
tdb_next = tdb + r*h*ftf/I + r*h*ftc/I

# ball position update
xb_next = xb + h*xdb_next
yb_next = yb + h*ydb_next
tb_next = tb + h*tdb_next

# floor velocity update
xdf_next = xdf + h*xd2f
ydf_next = ydf + h*yd2f

# floor position update
xf_next = xf + h*xdf_next
yf_next = yf + h*ydf_next

# state update
x_next = sp.Matrix([
    xb_next, yb_next, tb_next,
    xf_next, yf_next,
    xdb_next, ydb_next, tdb_next,
    xdf_next, ydf_next
])

# affine system
S0 = AffineSystem.from_symbolic(x, u, x_next)

In [5]:
# relative tangential velocity
sliding_velocity_floor = xdb_next + r*tdb_next - xdf_next
sliding_velocity_ceiling = xdb_next - r*tdb_next

# gap function floor and ceiling
gap_floor = yb_next - yf_next
gap_ceiling = d - 2.*r - yb_next

# gap function floor and ceiling in case of free dynamics
f_zero = {ftf: 0., fnf: 0., ftc: 0., fnc: 0.}
gap_floor_zero_force = gap_floor.subs(f_zero)
gap_ceiling_zero_force = gap_ceiling.subs(f_zero)

# ball distance to boundaries
ball_on_floor = sp.Matrix([
    xb_next - xf_next - l,
    xf_next - xb_next - l
])
ball_on_ceiling = sp.Matrix([
    xb_next - l,
    - xb_next - l
])

In [6]:
# state bounds
x_max = np.array([
    l, d-2.*r, 1.2*np.pi, # ball config
    l, l/2.,            # floor config
    2., 2., 10.,     # ball vel
    2., 2.          # floor vel
])
x_min = - x_max

# input bounds
u_max = np.array([
    30., 30.,       # floor acc
    100., 100.,       # floor force
    100., 100.        # ceiling force
])
u_min = - u_max

# domain bounds
xu = x.col_join(u)
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

In [7]:
# discrete time dynamics in mode 1
# (ball in the air)

# build domain
D1 = Polyhedron.from_bounds(xu_min, xu_max)

# set floor and ceiling forces to zero
for f in [ftf, fnf, ftc, fnc]:
    D1.add_symbolic_equality(xu, sp.Matrix([f]))

# - gap <= 0 with floor and ceiling
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_zero_force]))
D1.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_zero_force]))

# check domain
assert D1.bounded
assert not D1.empty

In [8]:
# discrete time dynamics in mode 2
# (ball sticking with the floor, not in contact with the ceiling)

# build domain
D2 = Polyhedron.from_bounds(xu_min, xu_max)

# set ceiling forces to zero
for f in [ftc, fnc]:
    D2.add_symbolic_equality(xu, sp.Matrix([f]))

# enforce sticking
D2.add_symbolic_equality(xu, sp.Matrix([sliding_velocity_floor]))
D2.add_symbolic_equality(xu, sp.Matrix([gap_floor]))

# gap <= 0 with floor
D2.add_symbolic_inequality(xu, sp.Matrix([gap_floor_zero_force]))

# - gap <= 0 with ceiling
D2.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_zero_force]))

# ball not falling down the floor
D2.add_symbolic_inequality(xu, ball_on_floor)

# friction cone
D2.add_symbolic_inequality(xu, sp.Matrix([ftf - mu*fnf]))
D2.add_symbolic_inequality(xu, sp.Matrix([- ftf - mu*fnf]))

# check domain
assert D2.bounded
assert not D2.empty

In [9]:
# discrete time dynamics in mode 3
# (ball sliding right on the floor, not in contact with the ceiling)

# build domain
D3 = Polyhedron.from_bounds(xu_min, xu_max)

# set ceiling forces to zero
for f in [ftc, fnc]:
    D3.add_symbolic_equality(xu, sp.Matrix([f]))
    
# enforce sliding
D3.add_symbolic_equality(xu, sp.Matrix([gap_floor]))
D3.add_symbolic_equality(xu, sp.Matrix([ftf + mu*fnf]))

# gap <= 0 with floor
D3.add_symbolic_inequality(xu, sp.Matrix([gap_floor_zero_force]))

# - gap <= 0 with ceiling
D3.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_zero_force]))

# ball not falling down the floor
D3.add_symbolic_inequality(xu, ball_on_floor)

# positive relative velocity
D3.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_floor]))

# check domain
assert D3.bounded
assert not D3.empty

In [10]:
# discrete time dynamics in mode 4
# (ball sliding left on the floor, not in contact with the ceiling)

# build domain
D4 = Polyhedron.from_bounds(xu_min, xu_max)

# set ceiling forces to zero
for f in [ftc, fnc]:
    D4.add_symbolic_equality(xu, sp.Matrix([f]))
    
# enforce sliding
D4.add_symbolic_equality(xu, sp.Matrix([gap_floor]))
D4.add_symbolic_equality(xu, sp.Matrix([ftf - mu*fnf]))

# gap <= 0 with floor
D4.add_symbolic_inequality(xu, sp.Matrix([gap_floor_zero_force]))

# - gap <= 0 with ceiling
D4.add_symbolic_inequality(xu, sp.Matrix([- gap_ceiling_zero_force]))

# ball not falling down the floor
D4.add_symbolic_inequality(xu, ball_on_floor)

# negative relative velocity
D4.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_floor]))

# check domain
assert D4.bounded
assert not D4.empty

In [11]:
# discrete time dynamics in mode 5
# (ball sticking on the ceiling, not in contact with the floor)

# build domain
D5 = Polyhedron.from_bounds(xu_min, xu_max)

# set floor forces to zero
for f in [ftf, fnf]:
    D5.add_symbolic_equality(xu, sp.Matrix([f]))

# enforce sticking
D5.add_symbolic_equality(xu, sp.Matrix([sliding_velocity_ceiling]))
D5.add_symbolic_equality(xu, sp.Matrix([gap_ceiling]))

# - gap <= 0 with floor
D5.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_zero_force]))

# gap <= 0 with ceiling
D5.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_zero_force]))

# ball in contact with the ceiling
D5.add_symbolic_inequality(xu, ball_on_ceiling)

# friction cone
D5.add_symbolic_inequality(xu, sp.Matrix([ftc - mu*fnc]))
D5.add_symbolic_inequality(xu, sp.Matrix([- ftc - mu*fnc]))

# check domain
assert D5.bounded
assert not D5.empty

In [12]:
# discrete time dynamics in mode 6
# (ball sliding right on the ceiling, not in contact with the floor)

# build domain
D6 = Polyhedron.from_bounds(xu_min, xu_max)

# set floor forces to zero
for f in [ftf, fnf]:
    D6.add_symbolic_equality(xu, sp.Matrix([f]))

# enforce sliding
D6.add_symbolic_equality(xu, sp.Matrix([gap_ceiling]))
D6.add_symbolic_equality(xu, sp.Matrix([ftc + mu*fnc]))

# - gap <= 0 with floor
D6.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_zero_force]))

# gap <= 0 with ceiling
D6.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_zero_force]))

# ball in contact with the ceiling
D6.add_symbolic_inequality(xu, ball_on_ceiling)

# positive relative velocity
D6.add_symbolic_inequality(xu, sp.Matrix([- sliding_velocity_ceiling]))

# check domain
assert D6.bounded
assert not D6.empty

In [13]:
# discrete time dynamics in mode 7
# (ball sliding left on the ceiling, not in contact with the floor)

# build domain
D7 = Polyhedron.from_bounds(xu_min, xu_max)

# set floor forces to zero
for f in [ftf, fnf]:
    D7.add_symbolic_equality(xu, sp.Matrix([f]))

# enforce sliding
D7.add_symbolic_equality(xu, sp.Matrix([gap_ceiling]))
D7.add_symbolic_equality(xu, sp.Matrix([ftc - mu*fnc]))

# - gap <= 0 with floor
D7.add_symbolic_inequality(xu, sp.Matrix([- gap_floor_zero_force]))

# gap <= 0 with ceiling
D7.add_symbolic_inequality(xu, sp.Matrix([gap_ceiling_zero_force]))

# ball in contact with the ceiling
D7.add_symbolic_inequality(xu, ball_on_ceiling)

# negative relative velocity
D7.add_symbolic_inequality(xu, sp.Matrix([sliding_velocity_ceiling]))

# check domain
assert D7.bounded
assert not D7.empty

In [14]:
# list of dynamics
S_list = [S0]*7

# list of domains
D_list = [D1, D2, D3, D4, D5, D6, D7]

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

In [15]:
# controller parameters
N = 20
Q = np.diag([
    1., .0, .0,
    1., 1.,
    1., 1., .01,
    1., 1.
])*h
R = np.diag([
    .01, .001,
    .0, .0,
    .0, .0
])*h
P = np.zeros((S.nx, S.nx))

# terminal set and cost
X_N = Polyhedron.from_bounds(*[np.zeros(S.nx)]*2)
# X_N = Polyhedron.from_bounds(x_min, x_max)

In [16]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull, lifted constraints')

In [17]:
# initial condition
x0 = np.array([
    0., 0., np.pi,
    0., 0.,
    0., 0., 0.,
    0., 0.
])
# x0 = np.array([
#     0., 0.2, 0.,
#     0., 0.,
#     0., 0., 0.,
#     0., 0.
# ])

In [18]:
controller.solve_relaxation(x0, {})[1]

0.10185444909674216

In [19]:
# controller.prog.setParam('MIPFocus', 2)
u_opt, x_opt, ms_opt, cost_opt = controller.feedforward(x0)

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 12070 rows, 6560 columns and 35450 nonzeros
Model has 140 quadratic constraints
Variable types: 6420 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+02]
  QMatrix range    [5e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 8110 rows and 4605 columns
Presolve time: 0.07s
Presolved: 3960 rows, 1955 columns, 13257 nonzeros
Variable types: 1824 continuous, 131 integer (131 binary)

Root relaxation: objective 0.000000e+00, 3355 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4          -    0.00000      -     -    0s
     0     0    0.00000    0   29          -    0.00000      -     -    0s
     0     0    

from pympc.control.hybrid_benchmark.branch_and_bound import branch_and_bound, best_first, depth_first
def solver(identifier, objective_cutoff):
    return controller.solve_relaxation(x0, identifier, objective_cutoff)
lb = {}
ub = {}
sol = {}
methods = [
    'Convex hull, lifted constraints',
    'Convex hull',
    'Big-M',
    'Traditional formulation'
]
for method in methods:
    print method
    controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method)
    sol_i, lb_i, ub_i = branch_and_bound(solver, depth_first, controller.explore_in_chronological_order)
    lb[method] = lb_i
    ub[method] = ub_i
    sol[method] = sol_i

In [ ]:
for i in range(S.nx):
    print 'x_'+str(i), sum(xt[i]*Q[i,i]*xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u_'+str(i), sum(ut[i]*R[i,i]*ut[i] for ut in u_opt)

In [ ]:
for i in range(S.nx):
    print 'x max_'+str(i), x_max[i], max(xt[i] for xt in x_opt)
    print 'x min_'+str(i), x_min[i], min(xt[i] for xt in x_opt)

for i in range(S.nu):
    print 'u max_'+str(i), u_max[i], max(ut[i] for ut in u_opt)
    print 'u min_'+str(i), u_min[i], min(ut[i] for ut in u_opt)

In [ ]:
plot_input_sequence(u_opt, h, (u_min, u_max))

In [ ]:
plot_state_trajectory(x_opt, h, (x_min, x_max))

for i in range(S.nx):
    print i, "min", min(xt[i] for xt in x)
    print i, "max", max(xt[i] for xt in x)
print x_min
print x_max

for i in range(S.nx):
    print sum(xt[i]*Q[i,i]*xt[i] for xt in x)
for i in range(S.nu):
    print sum(ut[i]*R[i,i]*ut[i] for ut in u)

In [ ]:
x0 = np.array([
    0., 0., 0.,
    0., 0.,
    0., 0., 0.,
    0., 0.
])
u_sim = [np.zeros(S.nu)]*50
x_sim, ms_sim = S.simulate(x0, u_sim)
print ms_sim

# Animation

In [ ]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

In [ ]:
tickness = .05
depth = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['ball'].set_object(
    Sphere(r),
    MeshLambertMaterial(color=blue)
)
vis['floor'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ceiling'].set_object(
    Box([depth, l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ball_orientation'].set_object(
    Cylinder(r/10., r),
    MeshLambertMaterial(color=green)
)

In [ ]:
anim = Animation()
for t, xt in enumerate(x_opt):
    with anim.at_frame(vis, t*h*300) as frame:
        frame['ball'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r])
        )
        frame['floor'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-tickness/2.])
        )
        frame['ceiling'].set_transform(
            tf.translation_matrix([0, 0, d+tickness/2.])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)

In [ ]:
D2.contains(np.zeros(S0.nx+S0.nu))

In [ ]:
x_opt[n]

In [ ]:
gap_ceiling_zero_force

In [ ]:
0.14670827 - 0.05*1.56583469 + 0.225

In [ ]:
ms_opt